# VROOM testing without ORS

updated with vroom1.8.0 vroom-express 0.7, supporting Shipments

docs: https://github.com/VROOM-Project/vroom/blob/master/docs/API.md#shipments

In [56]:
import openrouteservice as ors
import requests
import folium
import json
import random
from pprint import pprint

# DATI json posizione jobs (Bidoni)
pointsA = json.load(open('./data/mezzolombardo.geojson'))['features'] #4 points
pointsB = json.load(open('./data/mezzocorona.geojson'))['features'] #6 points

vroom_url = 'http://vroom:3000/optimization/'

center = [46.2147, 11.1065]
zoom = 15

# POSIZIONI PARTENZA DEI VEICOLI (Furgoni)
point_start = [46.2170, 11.1066]

# POSIZIONE ARRIVO DEI VEICOLI
point_end = [point_start[0], point_start[1]]

ccA = [p['geometry']['coordinates'] for p in pointsA]
ccB = [p['geometry']['coordinates'] for p in pointsB]
#ELENCO BIDONI 4 Mezzolombardo e 6 Mezzocorona TOTALE 10 con CONTENUTO CASUALE
job_locs = ccA + ccB

#colori veicoli/percorsi by vid
Vcolors = {
    '1': 'blue',
    '2': 'red'
}

# BUILD MAP from DATA

Map = folium.Map(location=center, tiles='OpenStreetMap', zoom_start=zoom)

vehicles = []

vid = 1
capacity = 10
vehicles.append({
    'id': vid,
    'capacity': [capacity],  # Limite di capacita' del Veicolo    
    'profile': 'driving-car',
    'start': list(reversed(point_start)), # posizione di partenza veicoli
    'end': list(reversed(point_start))
})
markerText = "capacity: %s" % capacity
mark = folium.Marker(point_start, tooltip=folium.Tooltip(markerText,permanent=True), icon=folium.Icon(icon='truck', prefix='fa', color=Vcolors[str(vid)]))
mark.add_to(Map)

point_startB = [point_start[0] + 0.0015, point_start[1] - 0.0015]

vid = 2
capacity = 10
vehicles.append({
    'id': vid,
    'capacity': [capacity],  # Limite di capacita' del Veicolo    
    'profile': 'driving-car',
    'start': list(reversed(point_startB)), # posizione di partenza veicoli
    'end': list(reversed(point_startB))
})
markerText = "capacity: %s" % capacity
mark = folium.Marker(point_startB, tooltip=folium.Tooltip(markerText,permanent=True), icon=folium.Icon(icon='truck', prefix='fa', color=Vcolors[str(vid)]))
mark.add_to(Map)

jobs = []
for idx, cc in enumerate(job_locs):
    loc = list(reversed(cc))
    
    delivery = random.randrange(1, 5) #quantita da consegnare
    service = random.randrange(60, 300) #job time service duration, da 1 a 5 minuti
    
    markerText = "quantity: %s <br>time: %s min" % (delivery, round(service/60))
    mark = folium.Marker(loc, tooltip=folium.Tooltip(markerText, permanent=True), icon=folium.Icon(icon='archive', prefix='fa', color='green'))
    mark.add_to(Map)

    jobs.append({
        'id': idx,
        'location': cc,
        'delivery': [delivery], # Quantita' della consegna
        'service': [service],
    })

Map

In [58]:
params = {
    'vehicles': vehicles,
    'jobs': jobs,
    'geometry': True
}
resp = requests.post(vroom_url, json = params)

solution = resp.json()

#solution['summary']
resp

<Response [400]>

In [41]:
for route in solution['routes']:
    poly = folium.PolyLine(
        locations = [list(reversed(coords)) for coords in ors.convert.decode_polyline(route['geometry'])['coordinates']],
        color = Vcolors[str(route['vehicle'])],
        tooltip = folium.Tooltip('Distance: '+str(solutionA['distance']/1000)+'km', permanent=True)
    )

    poly.add_to(Map)

Map

In [36]:
solution

{'code': 0,
 'summary': {'cost': 987,
  'unassigned': 3,
  'delivery': [0],
  'amount': [0],
  'pickup': [10],
  'service': 0,
  'duration': 987,
  'waiting_time': 0,
  'priority': 0,
  'distance': 10146,
  'computing_times': {'loading': 19, 'solving': 8, 'routing': 26}},
 'unassigned': [{'id': 6, 'location': [11.123356819152832, 46.21419929496722]},
  {'id': 2, 'location': [11.09168529510498, 46.21057627744751]},
  {'id': 4, 'location': [11.112627983093262, 46.21758452155989]}],
 'routes': [{'vehicle': 1,
   'cost': 475,
   'delivery': [0],
   'amount': [0],
   'pickup': [5],
   'service': 0,
   'duration': 475,
   'waiting_time': 0,
   'priority': 0,
   'distance': 5710,
   'steps': [{'type': 'start',
     'location': [11.1066, 46.217],
     'load': [0],
     'arrival': 0,
     'duration': 0,
     'distance': 0},
    {'type': 'job',
     'location': [11.118206977844238, 46.214911991592125],
     'id': 5,
     'service': 0,
     'waiting_time': 0,
     'job': 5,
     'load': [2],
    